<a href="https://colab.research.google.com/github/bhargavi5e1/bhargavi/blob/main/Number_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Number Recognition :
# Handwritten digit recognition system not only detects scanned images of handwritten digits.Handwritten digit recognition using MNIST dataset is a major project made with the help of Neural Network. It basically detects the scanned images of handwritten digits.

In [1]:
import numpy as np
import pandas as pd
import random

In [4]:
train = pd.read_csv('/content/train.csv.zip')
test = pd.read_csv('/content/test.csv.zip')
predicted_results = pd.read_csv('/content/sample_submission.csv')

In [5]:
train_X = train.drop('label', axis=1)/255
train_y = train['label']

samples = test/255

samples.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_X, train_y, test_size=.20, random_state=1)
print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

number of test samples : 8400
number of training samples: 33600


In [7]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

In [8]:
training_inputs = []
test_inputs = []
sample_inputs = []

for row in x_train.iterrows():
    training_inputs.append((row[1].values.reshape((784,1))))

training_results = [vectorized_result(y) for y in y_train]
training_data = list(zip(training_inputs, training_results))

for row in x_test.iterrows():
    test_inputs.append((row[1].values.reshape((784,1))))

test_data = list(zip(test_inputs, y_test))
for row in samples.iterrows():
    sample_inputs.append((row[1].values.reshape((784,1))))

In [13]:
class Network(object):
    def __init__(self,sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x,y in zip(sizes[:-1], sizes[1:])]

    def sigmoid(z):
        return 1.0/(1.0+np.exp(-z))

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        if test_data:
            n_test = sum(1 for _ in test_data)
        n = sum(1 for _ in training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {}: {} / {} --> {:2f}%".format(j, self.evaluate(test_data), n_test,(100*self.evaluate(test_data)/n_test)))
            else:
                print ("Epoch {} complete".format(j))
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)] #new weights
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)] #new biases
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases] # layer by layer vector of biases
        nabla_w = [np.zeros(w.shape) for w in self.weights]# layer by layer vector of weights
    # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (activations[-1]-y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        #l is negative index of list since it is backpropagation
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
      test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data] #how does generated test result match up with actual
      return sum(int(x == y) for (x, y) in test_results)

    def predict(self, samples):
        predicts = [(np.argmax(self.feedforward(x))) for x in samples] #generate results
        return predicts

def sigmoid(z): #build the sigmoid function for output transformation
        return 1.0/(1.0+np.exp(-z))
def sigmoid_prime(z):
    #Derivative of the sigmoid function
    return sigmoid(z)*(1-sigmoid(z))

**Generating the model itself using the defined class**

In [11]:
mindbrain = Network([784,64,64,10])

In [14]:
mindbrain.SGD(training_data, 30, 7, 1.2, test_data=test_data)

Epoch 0: 7913 / 8400 --> 94.202381%
Epoch 1: 7946 / 8400 --> 94.595238%
Epoch 2: 7936 / 8400 --> 94.476190%
Epoch 3: 7940 / 8400 --> 94.523810%
Epoch 4: 7932 / 8400 --> 94.428571%
Epoch 5: 7930 / 8400 --> 94.404762%
Epoch 6: 7958 / 8400 --> 94.738095%
Epoch 7: 7934 / 8400 --> 94.452381%
Epoch 8: 7959 / 8400 --> 94.750000%
Epoch 9: 7977 / 8400 --> 94.964286%
Epoch 10: 7973 / 8400 --> 94.916667%
Epoch 11: 7987 / 8400 --> 95.083333%
Epoch 12: 7979 / 8400 --> 94.988095%
Epoch 13: 7986 / 8400 --> 95.071429%
Epoch 14: 7993 / 8400 --> 95.154762%
Epoch 15: 7983 / 8400 --> 95.035714%
Epoch 16: 8004 / 8400 --> 95.285714%
Epoch 17: 8000 / 8400 --> 95.238095%
Epoch 18: 7995 / 8400 --> 95.178571%
Epoch 19: 8003 / 8400 --> 95.273810%
Epoch 20: 7996 / 8400 --> 95.190476%
Epoch 21: 7994 / 8400 --> 95.166667%
Epoch 22: 7997 / 8400 --> 95.202381%
Epoch 23: 7998 / 8400 --> 95.214286%
Epoch 24: 8001 / 8400 --> 95.250000%
Epoch 25: 8007 / 8400 --> 95.321429%
Epoch 26: 7994 / 8400 --> 95.166667%
Epoch 27: 7

**Predict the results**

In [15]:
results = mindbrain.predict(sample_inputs)

In [16]:
print('Results, frequency per number: \n')
print('0:', results.count(0))
print('1:', results.count(1))
print('2:', results.count(2))
print('3:', results.count(3))
print('4:', results.count(4))
print('5:', results.count(5))
print('6:', results.count(6))
print('7:', results.count(7))
print('8:', results.count(8))
print('9:', results.count(9))

Results, frequency per number: 

0: 2783
1: 3222
2: 2816
3: 2807
4: 2788
5: 2511
6: 2756
7: 2897
8: 2712
9: 2708


**Fill in the submissions table**

In [17]:
predicted_results.Label = results

In [18]:
predicted_results.head(20)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


**Generate the output**

In [19]:
predicted_results.to_csv('submission.csv', index=False)